In [ ]:
!pip install -U nltk

In [ ]:
import pandas as pd
import re
import json
from nltk.corpus import stopwords
import nltk
import numpy as np
nltk.download('stopwords')

from nltk.util import pad_sequence
from nltk.util import ngrams
from nltk import word_tokenize, sent_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
load_paths = ['../../data/train_data.csv', '../../data/valid_data.csv', '../../data/test_data.csv']
# load_paths2 = ['./train_data.csv', './valid_data.csv', './test_data.csv']

train_data = pd.read_csv(load_paths[0])
valid_data = pd.read_csv(load_paths[1])
test_data = pd.read_csv(load_paths[2])

In [ ]:
def filter_unwanted_rows(data):
    y = data['Label'].tolist()
    to_drop = [i for i in range(len(y)) if y[i] > 3]
    new_data = data.drop(labels=to_drop, axis=0)
    return np.asarray(new_data['Source']), np.asarray(new_data['Label'])

In [ ]:
X_train, y_train = filter_unwanted_rows(train_data)
X_test, y_test = filter_unwanted_rows(test_data)
X_valid, y_valid = filter_unwanted_rows(valid_data)

In [ ]:
from collections import Counter

print("Train labels ratio")
print(Counter(y_train.tolist()))
print("Test labels ratio")
print(Counter(y_test.tolist()))
print("Validation labels ratio")
print(Counter(y_valid.tolist()))

Train labels ratio
Counter({3: 14021, 2: 1942, 1: 65})
Test labels ratio
Counter({3: 3017, 2: 428, 1: 12})
Validation labels ratio
Counter({3: 3005, 2: 443, 1: 9})


In [ ]:
# Different techniques for tackling class imbalance
from imblearn.over_sampling import RandomOverSampler, SMOTE

def balance_data(x, y, _type=0):
    if _type == 1:
        ros = RandomOverSampler(random_state=42)
        return ros.fit_resample(x, y)
    elif _type == 2:
        smote = SMOTE()
        return smote.fit_resample(x, y)
    return x, y

In [ ]:
X_train, y_train = balance_data(X_train.reshape(-1, 1), y_train.reshape(-1, 1), 1)
print(Counter(y_train))

Counter({3: 14021, 2: 14021, 1: 14021})


In [ ]:
def get_probability_features(text):
    pass

In [ ]:
def find_correlation(scores, y):
    pass

In [ ]:
def get_score(probabilities):
    pass

In [ ]:
def get_improved_score(probabilities):
    pass

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, precision_score, f1_score, recall_score
import matplotlib.pyplot as plt

statements = {0: "Without oversampling", 1: "With random oversampling", 2: "With SMOTE"}

def get_metrics(b, ytrue, ypred):
    print('\n\n')
    print(statements[b])
    acc = accuracy_score(ytrue, ypred)
    f1 = f1_score(ytrue, ypred, average='micro')
    prec = precision_score(ytrue, ypred, average='micro')
    rec = recall_score(ytrue, ypred, average='micro')
    ConfusionMatrixDisplay.from_predictions(ytrue, ypred)
    plt.show()
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, F1-score: {f1}')
    print('\n\n')

In [ ]:
# LM training
nltk.download('punkt')

tokenized_data = []
for text in X_train:
    tokenized_data += [word_tokenize(sent.lower()) for sent in sent_tokenize(text[0])]
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_data)
model = MLE(n)
model.fit(train_data, padded_sents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Basic functioning of LM
sent = list(ngrams(tokenized_data[0], n=n, pad_left=True, pad_right=True))
model.score('women', 'men and'.split())

0.7272727272727273